# Schnorr Signature with pre-defined p and q
## ADD SOME STUFF ABOUT WHY SCHNORR IS GOOD
In this example, we were given a *p* value with which to calculate our *q*.  
We're also provided a generator *a*.  
  
The *q* in this example is **not secure** as it is of the form *p - 1* which can easily be factored.  
This is used later in the Pohlig-Hellman example.
  
Typically, *q* should be of the form *(p - 1) / 2* or more commonly written *p = 2q + 1*

In [14]:
import os
import gmpy2
import hashlib

### First we define our variables. 
**Message** is the message which will be signed.  
**p** is a prime.  
**q** is typically a prime of the form *p = 2q + 1* but in this case was created to be insecure.  
**a** is a generator for the prime *p*.  
**bits** is used when we generate random numbers.

In [15]:
Message = "slifhwenjfnewnfefwelafafeflmealfm32mflm23pfm3p2mf3m"
p = 776013677466283374829664077979289607703797802248972134730822459666549512547789492942820732883494078514296773729111709915350000261085021001313159067
q = p - 1
a = 2
bits = 512

**Generate the random secret**  
For a Schnorr Signature we'll generate a secret random number. It must be smaller than *q* so we do a quick loop to ensure that is true.  
os.urandom is used as this is said to be cryptographically secure.

*Note: in this case generating a 512 bit random number and checking that it's smaller than q is taking longer than I'd like, but 512 is secure where smaller wouldn't be recommneded.*

In [16]:
secret = int.from_bytes(os.urandom(bits//8),byteorder='big')
while secret >= (q-1):
    secret = int.from_bytes(os.urandom(bits//8),byteorder='big')

**Calculate the public key**  
The public key is our generator *a* raised to the negative random secret *s* modulo *p*.  
**public-key = a^-s mod p**  
pow is built in but doesn't handle a negative exponent. Rather than calculate the multiplicitave inverse I used gmpy2.powmod.

In [17]:
pub = gmpy2.powmod(a, -secret, p)

### Generate the signature  
Now that the variables have all been defined we'll generate the signature.  
To do this we'll generate another random number *r*.  
Once *r* is generated we calculate **a^r mod p** similar to before.  
We'll append **a^r mod p** to our message and then hash as *e*.  
To complete the signature we calculate **(r + s*e) mod q**

In [18]:
def signature_generation(M,p,q,a,s,bits):
    #computing e as part of the signature
    #r is a random number less than q
    r = int.from_bytes(os.urandom(bits//8),byteorder='big')
    while r > (q-1):
        r = int.from_bytes(os.urandom(bits//8),byteorder='big')

    x = pow(a,r,p)

    hash_string = M + str(x)
    e = hashlib.sha256(hash_string.encode()).hexdigest()

    #computing y as second part of the signature
    y = (r + s*int(e,16)) % q

    return int(e,16),int(y)

In [19]:
(e,y) = signature_generation(Message, p, q, a, secret, bits)

### Verify the signature  
When Alice signs a file she publishes all the below to Bob:  
- Message
- y, the signed hash
- e, the hashed message and appended **a^r mod p**
- public key
- p
- q
- a, the generator
- the hashing algorithm used

Bob then calculates **a^y * public^e (mod p)**  
Below we simplify it by calculating **a^y mod p** times **public^e mod p** then taking the result mod p again.

In [20]:
bob_prime = (gmpy2.powmod(a,y,p) * gmpy2.powmod(pub,e,p)) % p

After that calculation the result is appended to the message end and hashed.  
If this result matches *e* from Alice then the signature is valid.

In [21]:
#Convert H(M+x_prime) for verification against H(M+x)
hash_string = Message + str(bob_prime)
verify = hashlib.sha256(hash_string.encode()).hexdigest()
verify = int(verify,16)

In [22]:
if verify == e:
    print('Success! You verified the signature!')
else:
    print('You failed!!')

Success! You verified the signature!
